In [3]:
%pip install edgartools pandas beautifulsoup4 lxml ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.


In [5]:
# 1. Import the library
from edgar import *

# 2. Tell the SEC who you are (required by SEC regulations)
set_identity("slawomir.siudek@onlex.net")  # Replace with your email

# 3. Find a company
company = Company("MSFT")  # Microsoft

# 4. Get company filings
filings = company.get_filings() 

# 5. Filter by form 
insider_filings = filings.filter(form="4")  # Insider transactions

# 6. Get the latest filing
insider_filing = insider_filings[0]

# 7. Convert to a data object
ownership = insider_filing.obj()
print(ownership)

# import pandas as pd
# import time
# from bs4 import BeautifulSoup
# import os

# # 1. Pobierz listę firm z S&P 500
# sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
# tickers = sp500["Symbol"].tolist()

# # 2. Lista firm z EDGAR z mapowaniem Ticker -> CIK
# companies = et.get_companies()
# companies['ticker'] = companies['ticker'].str.upper()
# ticker_to_cik = dict(zip(companies['ticker'], companies['cik_str']))

# # 3. Pobierz raporty 10-Q dla wybranych spółek
# results = []

# for ticker in tickers[:10]:  # Demo: pierwsze 10 spółek
#     cik = ticker_to_cik.get(ticker)
#     if not cik:
#         continue

#     print(f"Pobieranie 10-Q dla {ticker} (CIK={cik})")
#     try:
#         filings = et.get_filings(cik, form="10-Q", count=4)  # 2 lata = ~4 kwartalne

#         if filings.empty:
#             continue

#         # Użyj pierwszego (najnowszego) zgłoszenia
#         filing = filings.iloc[0]
#         docs = et.get_documents(filing['accession_number'])
#         html_docs = docs[docs['type'].str.contains("htm", case=False)]

#         if html_docs.empty:
#             continue

#         # Pobierz pierwszy dokument HTML
#         url = html_docs.iloc[0]['url']
#         html = et.get_filing_document(url)

#         soup = BeautifulSoup(html, "lxml")
#         text = soup.get_text(" ", strip=True)

#         # Szukaj uproszczonych danych
#         def find_amount(keyword):
#             idx = text.lower().find(keyword.lower())
#             if idx == -1:
#                 return None
#             snippet = text[idx:idx+200]
#             numbers = [s.replace(",", "").replace("$", "") for s in snippet.split() if s.replace(",", "").replace("$", "").replace(".", "").isdigit()]
#             return float(numbers[0]) if numbers else None

#         revenue = find_amount("Revenue")
#         net_income = find_amount("Net Income")
#         debt = find_amount("Total Debt") or find_amount("Long-term debt")

#         # Heurystyka decyzji
#         if revenue is None or net_income is None:
#             decision = "Brak danych"
#         elif net_income > 0 and revenue > 0 and (debt or 0) < revenue * 0.5:
#             decision = "Kup"
#         else:
#             decision = "Sprzedaj"

#         results.append({
#             "Ticker": ticker,
#             "CIK": cik,
#             "Revenue": revenue,
#             "Net Income": net_income,
#             "Debt": debt,
#             "Decyzja": decision
#         })

#     except Exception as e:
#         print(f"❌ Błąd dla {ticker}: {e}")
#         continue

#     time.sleep(0.5)  # throttle to avoid SEC rate limits

# # 4. Wyniki jako DataFrame
# df = pd.DataFrame(results)
# print(df)
# df.to_csv("analiza_sp500.csv", index=False)


╭──────────────────────────── Ownership Transactions (Sale)  ────────────────────────────╮
│ Insider:  Bradford L Smith                                                             │
│ Position: Vice Chair and President                                                     │
│ Company:  MICROSOFT CORP (MSFT)                                                        │
│ Date:     2025-05-02                                                                   │
│ Form:     Form 4                                                                       │
│                               Common Stock Transactions                                │
│                                                                                        │
│   Type             Code   Description         Shares   Price/Share             Value   │
│  ────────────────────────────────────────────────────────────────────────────────────  │
│   SALE              S     Open Market Sale    18,212       $435.31     $7,927,780.12   │

In [7]:

def get_income_dataframe(ticker:str):
    c = Company(ticker)
    filings = c.get_filings(form="10-K").latest(5)
    xbs = XBRL.from_filings(filings)
    income_statement = xbs.statements.income_statement()
    income_df = income_statement.to_dataframe()
    return income_df
    

def plot_revenue(ticker:str):
    income_df = get_income_dataframe(ticker)
    
    # Extract financial metrics
    net_income = income_df[income_df.concept == "us-gaap_NetIncomeLoss"][income_statement.periods].iloc[0]
    gross_profit = income_df[income_df.concept == "us-gaap_GrossProfit"][income_statement.periods].iloc[0]
    revenue = income_df[income_df.label == "Revenue"][income_statement.periods].iloc[0]
    
    # Convert periods to fiscal years for better readability
    periods = [pd.to_datetime(period).strftime('FY%y') for period in income_statement.periods]
    
    # Reverse the order so most recent years are last (oldest to newest)
    periods = periods[::-1]
    revenue_values = revenue.values[::-1]
    gross_profit_values = gross_profit.values[::-1]
    net_income_values = net_income.values[::-1]
    
    # Create a DataFrame for plotting
    plot_data = pd.DataFrame({
        'Revenue': revenue_values,
        'Gross Profit': gross_profit_values,
        'Net Income': net_income_values
    }, index=periods)
    
    # Convert to billions for better readability
    plot_data = plot_data / 1e9
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot the data as lines with markers
    plot_data.plot(kind='line', marker='o', ax=ax, linewidth=2.5)
    
    # Format the y-axis to show billions with 1 decimal place
    ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'${x:.1f}B'))
    
    # Add labels and title
    ax.set_xlabel('Fiscal Year')
    ax.set_ylabel('Billions USD')
    ax.set_title(f'{c.name} ({ticker}) Financial Performance')
    
    # Add a grid for better readability
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Add a source note
    plt.figtext(0.5, 0.01, 'Source: SEC EDGAR via edgartools', ha='center', fontsize=9)
    
    # Improve layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    
    return fig

plot_revenue('MSFT')

AttributeError: type object 'XBRL' has no attribute 'from_filings'